In [39]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as fun
from pyspark.sql.functions import col
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import pipeline

In [4]:
spark = SparkSession.builder.master("local[2]").appName("ml_example").getOrCreate()

24/10/24 16:46:42 WARN Utils: Your hostname, Ravikants-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.12.13.156 instead (on interface en0)
24/10/24 16:46:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/24 16:46:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/24 16:46:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/10/24 16:46:43 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/10/24 16:46:43 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [19]:
df = spark.read.csv("headbrain.csv", header=True)

In [21]:
df.printSchema()

root
 |-- Gender: string (nullable = true)
 |-- Age Range: string (nullable = true)
 |-- Head Size(cm^3): string (nullable = true)
 |-- Brain Weight(grams): string (nullable = true)



In [23]:
df.show(5)

+------+---------+---------------+-------------------+
|Gender|Age Range|Head Size(cm^3)|Brain Weight(grams)|
+------+---------+---------------+-------------------+
|     1|        1|           4512|               1530|
|     1|        1|           3738|               1297|
|     1|        1|           4261|               1335|
|     1|        1|           3777|               1282|
|     1|        1|           4177|               1590|
+------+---------+---------------+-------------------+
only showing top 5 rows



In [25]:
df.count(), len(df.columns)

(237, 4)

In [27]:
df.select("Head Size(cm^3)", "Brain Weight(grams)").describe().show()

+-------+------------------+-------------------+
|summary|   Head Size(cm^3)|Brain Weight(grams)|
+-------+------------------+-------------------+
|  count|               237|                237|
|   mean|3633.9915611814345|  1282.873417721519|
| stddev| 365.2614224198132| 120.34044578645734|
|    min|              2720|               1012|
|    max|              4747|                955|
+-------+------------------+-------------------+



In [49]:
df = df.select(*(col(c).cast("Integer") for c in df.columns))

In [51]:
df.printSchema()

root
 |-- Gender: integer (nullable = true)
 |-- Age Range: integer (nullable = true)
 |-- Head Size(cm^3): integer (nullable = true)
 |-- Brain Weight(grams): integer (nullable = true)



In [53]:
features = df.drop(*["Gender", "Age Range"])

In [55]:
features.show(5)

+---------------+-------------------+
|Head Size(cm^3)|Brain Weight(grams)|
+---------------+-------------------+
|           4512|               1530|
|           3738|               1297|
|           4261|               1335|
|           3777|               1282|
|           4177|               1590|
+---------------+-------------------+
only showing top 5 rows



In [57]:
X = features.drop("Brain Weight(grams)")

In [59]:
X.show(5)

+---------------+
|Head Size(cm^3)|
+---------------+
|           4512|
|           3738|
|           4261|
|           3777|
|           4177|
+---------------+
only showing top 5 rows



In [61]:
assembler = VectorAssembler(inputCols=X.columns, outputCol='features')

In [63]:
output = assembler.transform(features).select("features", "Brain Weight(grams)")

In [65]:
output.show(5)

+--------+-------------------+
|features|Brain Weight(grams)|
+--------+-------------------+
|[4512.0]|               1530|
|[3738.0]|               1297|
|[4261.0]|               1335|
|[3777.0]|               1282|
|[4177.0]|               1590|
+--------+-------------------+
only showing top 5 rows



In [67]:
regression = LinearRegression(featuresCol='features', labelCol='Brain Weight(grams)')
regression_model = regression.fit(output)

24/10/24 17:03:44 WARN Instrumentation: [2b6e2623] regParam is zero, which might cause numerical instability and overfitting.
24/10/24 17:03:44 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/10/24 17:03:44 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
24/10/24 17:03:44 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [69]:
regression_model.coefficients

DenseVector([0.2634])

In [71]:
regression_model.intercept

325.5734210494322

In [75]:
regression_model.summary.r2

0.6393117199570006

In [77]:
regression_model.summary.rootMeanSquaredError

72.12062137837093